# Project 22: Activity Recognition
## Authors: Alessandro Pomes, Simon Schmoll
## Objectives: Classification of 7 activities which are tracked with a Single Chest-Mounted Accelerometer
## What is done in the Notebook: The data is imported, processed and classified


## Importing the libraries


In [ ]:
import pandas as pd
import numpy as np
